In [29]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from datetime import datetime, timedelta

In [3]:
zomato = pd.read_csv('./data/zomato_preprocessed_2.csv')
zomato = zomato.assign(order_time=pd.to_datetime(zomato['order_time']))
zomato

,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,is_affordable_restaurant,is_affordable_voucher,is_affordable,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue
0,543991243,2021-09-01 00:00:51,ygyH_eaSHCTSSgAda33qDLYA,False,True,NaN,NaN,NaN,TGO,67.43,...,False,False,True,13.486,Chennai,53.944,False,20.0,False,5.44
1,543992115,2021-09-01 00:02:50,vagdCBP6L9S461DAPk0tNs-A,False,True,NaN,NaN,NaN,TGO,53.50,...,False,False,True,NaN,Mumbai,53.500,True,NaN,False,9.17
2,543992763,2021-09-01 00:04:26,65RUZmnykIRzyWaqnOaB_EdQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,72.99,...,False,False,False,NaN,Chennai,72.990,True,NaN,False,6.92
3,543993755,2021-09-01 00:06:41,THNssTsOurRqOJf9sQEhwiRg,False,True,NaN,NaN,NaN,TGO,206.00,...,False,False,False,NaN,Chennai,206.000,True,NaN,False,5.83
4,543994010,2021-09-01 00:07:13,OGX49Bsq0pR5qB32sT83JhSg,False,True,NaN,NaN,NaN,TGO,100.99,...,False,False,False,NaN,Chennai,100.990,True,NaN,False,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118275,644315564,2022-01-02 23:57:08,xdSbyNHiYORwiK2AHYieL5UQ,False,False,CUSTOMER_MISTAKE_ERROR,Changed mind - Mistake / Error,CUSTOMER,TGO,147.49,...,False,False,False,NaN,Chennai,147.490,True,NaN,False,6.62
118276,644315682,2022-01-02 23:57:25,AYeFPGmOUyT4OIWfmTSCYikw,False,True,NaN,NaN,NaN,TGO,110.49,...,False,False,False,NaN,Coimbatore,110.490,True,NaN,False,9.90
118277,644316048,2022-01-02 23:58:05,COnO0tda5iSTa-t5dFFVTpwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.49,...,False,False,False,NaN,Chennai,112.490,True,NaN,False,7.36
118278,644316064,2022-01-02 23:58:07,iI6rRPz2LkRi2Ct0zTaksrOA,False,True,NaN,NaN,NaN,TGO,739.24,...,False,False,False,NaN,Chennai,739.240,True,NaN,False,5.17


In [73]:
ressurected_list = []

for _, group in tqdm(zomato.groupby('analytical_customer_id')) : 
    temp = group.sort_values('order_time').reset_index(drop=True)
    temp = temp.assign(order_time_diff=temp['order_time'].diff())
    
    for i in range(temp.shape[0]) : 
        if ~temp.iloc[i]['is_successful'] : 
            try : 
                if (temp.iloc[i+1]['order_time_diff'] <= timedelta(minutes=30)) & (temp.iloc[i+1]['is_successful']) : 
                    temp.loc[temp.index==i, 'is_ressurected'] = True
                else :
                    temp.loc[temp.index==i, 'is_ressurected'] = False
            except : 
                pass
    temp = temp.drop('order_time_diff', axis=1)
    ressurected_list.append(temp)

100%|███████████████████████████████████████████████████████████████████████████| 13559/13559 [00:45<00:00, 295.79it/s]


In [74]:
zomato_ressurected = pd.concat(ressurected_list, axis=0).sort_values('order_time').reset_index(drop=True)
zomato_ressurected = zomato_ressurected.assign(is_ressurected=zomato_ressurected['is_ressurected'].fillna(False))
zomato_ressurected

,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,is_affordable_voucher,is_affordable,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected
0,543991243,2021-09-01 00:00:51,ygyH_eaSHCTSSgAda33qDLYA,False,True,NaN,NaN,NaN,TGO,67.43,...,False,True,13.486,Chennai,53.944,False,20.0,False,5.44,False
1,543992115,2021-09-01 00:02:50,vagdCBP6L9S461DAPk0tNs-A,False,True,NaN,NaN,NaN,TGO,53.50,...,False,True,NaN,Mumbai,53.500,True,NaN,False,9.17,False
2,543992763,2021-09-01 00:04:26,65RUZmnykIRzyWaqnOaB_EdQ,False,False,LATE_DELIVERY,NaN,VENDOR,TGO,72.99,...,False,False,NaN,Chennai,72.990,True,NaN,False,6.92,False
3,543993755,2021-09-01 00:06:41,THNssTsOurRqOJf9sQEhwiRg,False,True,NaN,NaN,NaN,TGO,206.00,...,False,False,NaN,Chennai,206.000,True,NaN,False,5.83,False
4,543994010,2021-09-01 00:07:13,OGX49Bsq0pR5qB32sT83JhSg,False,True,NaN,NaN,NaN,TGO,100.99,...,False,False,NaN,Chennai,100.990,True,NaN,False,7.41,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118275,644315564,2022-01-02 23:57:08,xdSbyNHiYORwiK2AHYieL5UQ,False,False,CUSTOMER_MISTAKE_ERROR,Changed mind - Mistake / Error,CUSTOMER,TGO,147.49,...,False,False,NaN,Chennai,147.490,True,NaN,False,6.62,False
118276,644315682,2022-01-02 23:57:25,AYeFPGmOUyT4OIWfmTSCYikw,False,True,NaN,NaN,NaN,TGO,110.49,...,False,False,NaN,Coimbatore,110.490,True,NaN,False,9.90,False
118277,644316048,2022-01-02 23:58:05,COnO0tda5iSTa-t5dFFVTpwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.49,...,False,False,NaN,Chennai,112.490,True,NaN,False,7.36,False
118278,644316064,2022-01-02 23:58:07,iI6rRPz2LkRi2Ct0zTaksrOA,False,True,NaN,NaN,NaN,TGO,739.24,...,False,False,NaN,Chennai,739.240,True,NaN,False,5.17,False


In [82]:
zomato_ressurected.columns

Index(['order_id', 'order_time', 'analytical_customer_id', 'is_acquisition',
       'is_successful', 'reason', 'sub_reason', 'owner',
       'delivery_arrangement', 'gmv_amount_lc', 'basket_amount_lc',
       'delivery_fee_amount_lc', 'Payment Method', 'actual_delivery_time',
       'promised_delivery_time', 'order_delay', 'dropoff_distance_manhattan',
       'platform', 'vertical_class', 'vertical', 'is_affordable_freedelivery',
       'is_affordable_item', 'is_affordable_gem', 'is_affordable_restaurant',
       'is_affordable_voucher', 'is_affordable', 'affordability_amt_total',
       'City', 'pay_amount', 'is_consistent', 'afford_ratio', 'is_gold',
       'ad_revenue', 'is_ressurected'],
      dtype='object')

In [99]:
zomato_ressurected.loc[zomato_ressurected['analytical_customer_id']=='aercO_SKOCTaK1DdSAHzcRwg'][['analytical_customer_id','order_time', 'is_successful', 'pay_amount',
                                                                                                  'basket_amount_lc', 'delivery_fee_amount_lc', 'vertical_class', 'is_ressurected']]

,analytical_customer_id,order_time,is_successful,pay_amount,basket_amount_lc,delivery_fee_amount_lc,vertical_class,is_ressurected
17,aercO_SKOCTaK1DdSAHzcRwg,2021-09-01 00:29:15,False,235.0,195.0,40.0,food,True
18,aercO_SKOCTaK1DdSAHzcRwg,2021-09-01 00:30:33,True,40.0,195.0,40.0,food,False
7860,aercO_SKOCTaK1DdSAHzcRwg,2021-09-09 00:44:09,True,10.0,202.0,40.0,food,False
10789,aercO_SKOCTaK1DdSAHzcRwg,2021-09-12 03:58:51,True,220.7,216.0,35.0,food,False
14616,aercO_SKOCTaK1DdSAHzcRwg,2021-09-16 02:48:13,True,154.0,135.0,19.0,food,False


In [75]:
ressurected = zomato_ressurected[zomato_ressurected['is_ressurected']==True]
ressurected

,order_id,order_time,analytical_customer_id,is_acquisition,is_successful,reason,sub_reason,owner,delivery_arrangement,gmv_amount_lc,...,is_affordable_voucher,is_affordable,affordability_amt_total,City,pay_amount,is_consistent,afford_ratio,is_gold,ad_revenue,is_ressurected
17,544003009,2021-09-01 00:29:15,aercO_SKOCTaK1DdSAHzcRwg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,235.00,...,False,False,NaN,Chennai,235.00,True,NaN,False,7.27,True
81,544035841,2021-09-01 02:38:50,iTjnN6CCYiTkCtoFCGkUh9Tg,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TMP,139.48,...,False,False,NaN,Delhi,139.48,True,NaN,False,6.49,True
96,544042740,2021-09-01 03:31:11,PFSLF6bmoLTUmlWQbjuaWC7A,False,False,NO_COURIER,NaN,VENDOR,TMP,90.00,...,False,False,NaN,Chennai,90.00,True,NaN,False,6.83,True
155,544146996,2021-09-01 10:44:38,TQCx8IzyX7Tgyt3azQHuY3wQ,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,39.19,...,False,False,NaN,Chennai,39.19,True,NaN,False,6.18,True
172,544164308,2021-09-01 11:15:43,w10UJYEexxS4eXraEqa9iJgQ,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,112.10,...,False,False,NaN,Chennai,112.10,True,NaN,False,5.02,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118240,644298082,2022-01-02 23:22:30,gGVPQfimlaSomX1oidrWYVpg,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TGO,70.99,...,False,False,NaN,Delhi,70.99,True,NaN,True,NaN,True
118245,644300790,2022-01-02 23:27:28,T3R-gTCSbiS2SPlJCXbrs_HA,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,229.02,...,False,False,NaN,Chennai,229.02,True,NaN,False,6.22,True
118252,644302401,2022-01-02 23:30:12,zMdSeVGGvOSaGOEsGuuh0a5Q,False,False,ONLINE_PAYMENT,NaN,CUSTOMER/PLATFORM,TMP,505.00,...,False,False,NaN,Chennai,505.00,True,NaN,True,NaN,True
118259,644304881,2022-01-02 23:35:13,yn81_8Q-pJTQ--7gQtrOjPvQ,False,False,CUSTOMER_MISTAKE_ERROR,Canceled by Customer - Before vendor accepted,CUSTOMER,TGO,113.99,...,False,False,NaN,Mumbai,113.99,True,NaN,False,5.74,True


In [77]:
# zomato_ressurected.to_csv('./data/zomato_preprocessed_3.csv', index=False)